In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset

dataset = load_dataset("DeepPavlov/d2g_generated", token=True)

In [5]:
import re

def match_greeting(message: dict):
    return bool(
        re.match(r'^hello|^hi',
                 message['text'],
                 flags=re.IGNORECASE
        )
    )

In [11]:
idx_duplicates = []
for i, ex in enumerate(dataset['train']):
    for j, dia in enumerate(ex['dialogues']):
        num_greeting = sum([match_greeting(mes) for mes in dia['messages']])
        if num_greeting > 1:
            idx_duplicates.append((i, j))

In [14]:
len(idx_duplicates)

221

In [19]:
idx_duplicates

[(2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 13),
 (7, 14),
 (7, 15),
 (7, 16),
 (17, 12),
 (17, 13),
 (17, 14),
 (17, 15),
 (17, 16),
 (17, 17),
 (20, 0),
 (20, 1),
 (20, 2),
 (30, 0),
 (30, 1),
 (30, 2),
 (30, 3),
 (30, 4),
 (30, 5),
 (30, 6),
 (32, 2),
 (51, 0),
 (51, 1),
 (51, 2),
 (51, 3),
 (51, 4),
 (51, 5),
 (51, 6),
 (51, 7),
 (51, 8),
 (51, 9),
 (51, 10),
 (51, 11),
 (51, 12),
 (51, 13),
 (51, 14),
 (51, 15),
 (51, 16),
 (51, 17),
 (65, 3),
 (65, 4),
 (65, 5),
 (75, 8),
 (79, 1),
 (79, 3),
 (84, 0),
 (84, 1),
 (84, 2),
 (84, 3),
 (84, 4),
 (84, 5),
 (84, 6),
 (84, 7),
 (84, 8),
 (92, 0),
 (92, 1),
 (92, 2),
 (92, 3),
 (92, 4),
 (92, 5),
 (92, 6),
 (95, 2),
 (95, 3),
 (95, 6),
 (117, 0),
 (117, 1

In [ ]:
class DuplicateDialogues:

    def __init__(self, data: list, 
                 idx_duplicates: list):
        self.data = data
        self.idx_duplicates = idx_duplicates

    def get_duplicate_dialogue(self, idx):
        idx_topic, idx_dialogue = idx_duplicates[idx]
        return self.data[idx_topic]['dialogues'][idx_dialogue]


In [32]:
dd = DuplicateDialogues(dataset['train'], idx_duplicates)

In [33]:
dd.get_duplicate_dialogue(0)

{'id': 'Assisting with chatbot limitations.Addressing fake social media accounts.Managing review responses on Google/Yelp._1_0',
 'messages': [{'participant': 'assistant',
   'text': 'Hello! How can I assist you today?'},
  {'participant': 'user', 'text': 'I need help with chatbot limitations.'},
  {'participant': 'assistant',
   'text': "Sure, could you please specify the chatbot limitations you're facing?"},
  {'participant': 'user', 'text': 'Actually, I want to change my topic.'},
  {'participant': 'assistant',
   'text': 'Of course! What would you like to discuss instead?'},
  {'participant': 'user', 'text': 'I have another question.'},
  {'participant': 'assistant', 'text': 'Hello! How can I assist you today?'},
  {'participant': 'user',
   'text': 'I need assistance managing my review responses on Google/Yelp.'},
  {'participant': 'assistant',
   'text': 'Certainly! Are you looking to respond to positive or negative reviews?'},
  {'participant': 'user', 'text': 'Positive reviews.